In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42663")
client

Client Scheduler: tcp://127.0.0.1:42663 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 14 Cores: 70 Memory: 296.33 GB


In [2]:
import numpy as np
import xarray as xr
import pandas as pd
from dask.diagnostics import ProgressBar
import os.path as op
import os
# import gc
# import h5py
import xrft
import gsw
import time
from scipy.interpolate import PchipInterpolator, interp1d
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from MITgcmutils import jmd95
from fastjmd95 import rho as densjmd95
from fastjmd95 import drhods, drhodt
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from xlayers import finegrid, layers
from xlayers.core import layers_numpy, layers_apply

In [4]:
ddir = '/tank/chaocean/'
savedir = '/tank/topog/tuchida/TWA/'

In [5]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
Cp = 3.994e3
ySstart = -15
yNstart = 10
yend = 50

Dsig = 30
nlayers = 60
siglayers = np.linspace(20,50,nlayers)
func = interp1d(range(1,nlayers+1), siglayers, fill_value='extrapolate')
sigp1layers = func(np.arange(.5,nlayers+1.5,1))

In [6]:
xchunk = 200
ychunk = 200
sigchunk = 10
membchunk = 1
dnf = 2160
nremap = 90

In [ ]:
# for itime in range(1380240+dnf,1416960+dnf,dnf):
for itime in range(943920,943920+dnf,dnf):
    
    for nmemb in range(24,36):     
        if nmemb == 24:
            ds = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                         % (nmemb,year)), 
                                 grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                 prefix=['diag_ocnSnap'], 
                                 delta_t=2e2, iters=itime, 
                                 chunks={'XC':xchunk,'XG':xchunk}
                                ).isel(time=-1)
        else:
            ds = xr.concat([ds, open_mdsdataset(op.join(ddir,
                                                        'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                                        % (nmemb,year)), 
                                                grid_dir=op.join(ddir,
                                                                 'grid_chaO/gridMIT_update1/'),
                                                prefix=['diag_ocnSnap'], 
                                                delta_t=2e2, iters=itime, 
                                                chunks={'XC':xchunk,'XG':xchunk}
                                               ).isel(time=-1)],
                            dim='nmemb')
    ds.coords['nmemb'] = ('nmemb',np.arange(0,12,1))
            
    fine_drf = np.squeeze(ds.drF.sel(Z=slice(None,None)))
    fine_drc = np.squeeze(ds.drC.sel(Zp1=slice(None,None)))
#     drf_finer,mapindex,mapfact,cellindex = finegrid.finegrid(fine_drf, fine_drc, nremap)

    grid = Grid(ds, periodic=['X'])
    
#################################
#     rho_insitu = xr.apply_ufunc(jmd95.densjmd95, 
#                                 ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
#                                 ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
# #                                 (ds.PHIHYD.sel(YC=slice(ySstart-12**-1,yend+2*12**-1))
#                                 -grav*ds.Z*rhoConst*1e-4, # pressure in [dbar]!!!!
#                                 dask='parallelized', output_dtypes=[float,]
#                                ).where(ds.maskC!=0.
#                                       ).compute().chunk({'nmemb':membchunk,
#                                                          'XC':xchunk,'YC':ychunk})
    
#     rhox = xr.apply_ufunc(jmd95.densjmd95, 
#                           grid.interp(ds.SALT,'X').sel(YC=slice(yNstart-12**-1,
#                                                                 yend+2*12**-1)), 
#                           grid.interp(ds.THETA,'X').sel(YC=slice(yNstart-12**-1,
#                                                                  yend+2*12**-1)), 
# #                           (grid.interp(ds.PHIHYD,'X').sel(YC=slice(ySstart-12**-1,
# #                                                                    yend+2*12**-1))
#                           -grav*ds.Z*rhoConst*1e-4, 
#                           dask='parallelized', output_dtypes=[float,]
#                          ).where(ds.maskW!=0.
#                                 ).compute().chunk({'nmemb':membchunk,
#                                                    'XG':xchunk,'YC':ychunk})
    
#     rhoy = xr.apply_ufunc(jmd95.densjmd95, 
#                           grid.interp(ds.SALT,'Y',boundary='fill'
#                                      ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
#                           grid.interp(ds.THETA,'Y',boundary='fill'
#                                      ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
# #                           (grid.interp(ds.PHIHYD,'Y',boundary='fill'
# #                                       ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1))
#                           -grav*ds.Z*rhoConst*1e-4,
#                           dask='parallelized', output_dtypes=[float,]
#                          ).where(ds.maskS!=0.
#                                 ).compute().chunk({'nmemb':membchunk,
#                                                    'XC':xchunk,'YG':ychunk})
    
#     rhoz = xr.apply_ufunc(jmd95.densjmd95, 
#                           grid.interp(grid.interp(ds.SALT,'Y',boundary='fill'),
#                                       'X').sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
#                           grid.interp(grid.interp(ds.THETA,'Y',boundary='fill'),
#                                       'X').sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
# #                           (grid.interp(grid.interp(ds.PHIHYD,'Y',boundary='fill'),'X'
# #                                       ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1))
#                           -grav*ds.Z*rhoConst*1e-4,
#                           dask='parallelized', output_dtypes=[float,]
#                          ).where(grid.interp(ds.maskS,'X')!=0.
#                                 ).compute().chunk({'nmemb':membchunk,
#                                                    'XG':xchunk,'YG':ychunk})

    rho_insitu = xr.apply_ufunc(densjmd95, 
                                ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                                ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
                                -grav*ds.Z*rhoConst*1e-4, # pressure in [dbar]!!!!
                                dask='parallelized', output_dtypes=[float,]
                               ).where(ds.maskC!=0.
                                      ).compute().chunk({'nmemb':membchunk,
                                                         'XC':xchunk,'YC':ychunk})
    
    rhox = xr.apply_ufunc(densjmd95, 
                          grid.interp(ds.SALT,'X').sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                          grid.interp(ds.THETA,'X').sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                          -grav*ds.Z*rhoConst*1e-4, 
                          dask='parallelized', output_dtypes=[float,]
                         ).where(ds.maskW!=0.
                                ).compute().chunk({'nmemb':membchunk,
                                                   'XG':xchunk,'YC':ychunk})
    
    rhoy = xr.apply_ufunc(densjmd95, 
                          grid.interp(ds.SALT,'Y',boundary='fill'
                                     ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          grid.interp(ds.THETA,'Y',boundary='fill'
                                     ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          -grav*ds.Z*rhoConst*1e-4,
                          dask='parallelized', output_dtypes=[float,]
                         ).where(ds.maskS!=0.
                                ).compute().chunk({'nmemb':membchunk,
                                                   'XC':xchunk,'YG':ychunk})
    
    rhoz = xr.apply_ufunc(densjmd95, 
                          grid.interp(grid.interp(ds.SALT,'Y',boundary='fill'),
                                      'X').sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          grid.interp(grid.interp(ds.THETA,'Y',boundary='fill'),
                                      'X').sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          -grav*ds.Z*rhoConst*1e-4,
                          dask='parallelized', output_dtypes=[float,]
                         ).where(grid.interp(ds.maskS,'X')!=0.
                                ).compute().chunk({'nmemb':membchunk,
                                                   'XG':xchunk,'YG':ychunk})
    
########################################
    start = time.time()
    zz, _ = xr.broadcast(ds.Z, rho_insitu.isel(Z=0
                                              ).sel(YC=slice(yNstart-12**-1,yend+2*12**-1))
                        )
    zzy, _ = xr.broadcast(ds.Z, rhoy.isel(Z=0
                                         ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1))
                         )
    zzx, _ = xr.broadcast(ds.Z, rhox.isel(Z=0
                                         ).sel(YC=slice(yNstart-12**-1,yend+2*12**-1))
                         )
    zz = zz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk}).transpose('nmemb','Z','YC','XC')
    zzy = zzy.chunk({'nmemb':membchunk,'YG':ychunk,'XC':xchunk}).transpose('nmemb','Z','YG','XC')
    zzx = zzx.chunk({'nmemb':membchunk,'YC':ychunk,'XG':xchunk}).transpose('nmemb','Z','YC','XG')
#     zzz, _ = xr.broadcast(ds.Z, rhoz.isel(Z=0,nmemb=0).sel(YG=slice(ySstart-12**-1,
#                                                                     yend+2*12**-1))
#                          )

#     zdz = xr.DataArray(np.ones((len(ds.nmemb),
#                                 len(ds.YC.sel(YC=slice(yNstart-12**-1,yend+2*12**-1))),
#                                 len(ds.XC),len(siglayers))),
#                        dims=['nmemb','YC','XC','sig'],
#                        coords={'nmemb':range(len(ds.nmemb),),
#                                'YC':ds.YC.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)).data,
#                                'XC':ds.XC.data,'sig':siglayers}
#                       ) * np.nan
#     dzetaF = zdz.copy()
#     zxdz = xr.DataArray(np.ones((len(ds.nmemb),
#                                 len(ds.YC.sel(YC=slice(yNstart-12**-1,yend+2*12**-1))),
#                                 len(ds.XG),len(siglayers))),
#                        dims=['nmemb','YC','XG','sig'],
#                        coords={'nmemb':range(len(ds.nmemb),),
#                                'YC':ds.YC.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)).data,
#                                'XG':ds.XG.data,'sig':siglayers}
#                        ) * np.nan
#     dzetaFx = zxdz.copy()
#     zydz = xr.DataArray(np.ones((len(ds.nmemb),
#                                 len(ds.YG.sel(YG=slice(yNstart-12**-1,yend+2*12**-1))),
#                                 len(ds.XC),len(siglayers))),
#                        dims=['nmemb','YG','XC','sig'],
#                        coords={'nmemb':range(len(ds.nmemb),),
#                                'YG':ds.YG.sel(YG=slice(yNstart-12**-1,yend+2*12**-1)).data,
#                                'XC':ds.XC.data,'sig':siglayers}
#                        ) * np.nan
#     dzetaFy = zydz.copy()
#     dzetaFz = xr.DataArray(np.ones((len(ds.nmemb),
#                                     len(ds.YG.sel(YG=slice(yNstart-12**-1,yend+2*12**-1))),
#                                     len(ds.XG),len(siglayers))),
#                            dims=['nmemb','YG','XG','sig'],
#                            coords={'nmemb':range(len(ds.nmemb),),
#                                    'YG':ds.YG.sel(YG=slice(yNstart-12**-1,yend+2*12**-1)).data,
#                                    'XG':ds.XG.data,'sig':siglayers}
#                           ) * np.nan
    
#     for tt in range(len(ds.nmemb)):
#         tmp1 = xr.apply_ufunc(layers_numpy, 
#                               xr.ones_like(zz).chunk({'XC':xchunk,'YC':ychunk}),
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tmp2 = xr.apply_ufunc(layers_numpy, 
#                               zz.chunk({'XC':xchunk,'YC':ychunk}), 
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         dzetaF[tt] = tmp1.data
#         zdz[tt] = tmp2.data
#         del tmp1, tmp2
    dzetaF = layers_apply(xr.ones_like(zz).chunk({'nmemb':membchunk,'XC':xchunk,'YC':ychunk}),
                          (rho_insitu-rhoConst),
                          siglayers, fine_drf, fine_drc, fine_fact=nremap,
                          lev_name='Z', Tlev_name='sig').compute()
    zdz = layers_apply(zz, (rho_insitu-rhoConst),
                       siglayers, fine_drf, fine_drc, fine_fact=nremap,
                       lev_name='Z', Tlev_name='sig').compute()
        
#         tmp3 = xr.apply_ufunc(layers_numpy, 
#                               xr.ones_like(zzx).chunk({'XG':xchunk,'YC':ychunk}),
#                               (rhox.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tmp4 = xr.apply_ufunc(layers_numpy, 
#                               zzx.chunk({'XG':xchunk,'YC':ychunk}), 
#                               (rhox.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         dzetaFx[tt] = tmp3.data
#         zxdz[tt] = tmp4.data
#         del tmp3, tmp4
    dzetaFx = layers_apply(xr.ones_like(zzx).chunk({'nmemb':membchunk,'XG':xchunk,'YC':ychunk}),
                           (rhox-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
    zxdz = layers_apply(zzx, (rhox-rhoConst),
                        siglayers, fine_drf, fine_drc, fine_fact=nremap,
                        lev_name='Z', Tlev_name='sig').compute()
        
#         tmp5 = xr.apply_ufunc(layers_numpy, 
#                               xr.ones_like(zzy).chunk({'XC':xchunk,'YG':ychunk}),
#                               (rhoy.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tmp6 = xr.apply_ufunc(layers_numpy, 
#                               zzy.chunk({'XC':xchunk,'YG':ychunk}), 
#                               (rhoy.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         dzetaFy[tt] = tmp5.data
#         zydz[tt] = tmp6.data
#         del tmp5, tmp6
    dzetaFy = layers_apply(xr.ones_like(zzy).chunk({'nmemb':membchunk,'XC':xchunk,'YG':ychunk}),
                           (rhoy-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
    zydz = layers_apply(zzy, (rhoy-rhoConst),
                        siglayers, fine_drf, fine_drc, fine_fact=nremap,
                        lev_name='Z', Tlev_name='sig').compute()
        
#         tmp7 = xr.apply_ufunc(layers_numpy, 
#                               xr.ones_like(rhoz.isel(nmemb=tt)).chunk({'XG':xchunk,
#                                                                        'YG':ychunk}),
#                               (rhoz.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized', 
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

# #         tmp8 = xr.apply_ufunc(layers_numpy, 
# #                               zzxy.chunk({'XC':xchunk}),
# #                               (rhoxy.isel(nmemb=tt)-rhoConst).sel(YG=slice(ystart-12**-1,
# #                                                                            yend+2*12**-1)),
# #                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
# #                                       'mapindex':mapindex,'cellindex':cellindex,
# #                                       'drf_finer':drf_finer},
# #                               dask='parallelized', 
# #                               input_core_dims=[['Z'],['Z']], 
# #                               output_core_dims=[['sig']],
# #                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
# #                              ).compute()

#         dzetaFz[tt] = tmp7.data
# #         zxydz[tt] = tmp8.data
#         del tmp7
    dzetaFz = layers_apply(xr.ones_like(rhoz).chunk({'nmemb':membchunk,'XG':xchunk,'YG':ychunk}),
                           (rhoz-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
    
    del rho_insitu, rhoz, rhoy, rhox
    del zz, zzy, zzx
    end = time.time()
    print("Interpolation done. Lapse:", end-start)
    
    
##########################################
    dsave = zdz.to_dataset(name='zdz')
    dsave['zydz'] = zydz
    dsave['zxdz'] = zxdz
    dsave['dzetaF'] = dzetaF
    dsave['dzetaFx'] = dzetaFx
    dsave['dzetaFy'] = dzetaFy
    dsave['dzetaFz'] = dzetaFz
    dsave.coords['sigp1'] = ('sigp1',sigp1layers)
    dsave.to_netcdf(op.join(savedir,'%4d/Zs_membs_parallel-%2d_%010d.nc' % (year,nremap,itime)))
    dsave.close()
    ds.close()
    del zdz, zydz, zxdz, dzetaF, dzetaFz, dzetaFy, dzetaFx
#     client.restart()
    
    print(itime)

In [7]:
def sound_speed(s,th,p):
    """
    s            : salinity (psu)
    th           : potential temperature (deg C, ITS-90)
    p            : pressure (dbar)
    """
    th2 = th**2
    sqrts = np.sqrt(s)

    anum = ( 9.99843699e2 +
            th *( 7.35212840 +
                 th *(-5.45928211e-2 +
                      th * 3.98476704e-4
                     )
                ) +
            s *( 2.96938239 -
                th * 7.23268813e-3 +
                s * 2.12382341e-3
               )
           )
    aden = (1. +
            th *( 7.28606739e-3 +
                 th *(-4.60835542e-5 +
                      th *( 3.68390573e-7 +
                           th * 1.80809186e-10
                          )
                     )
                ) +
            s *( 2.14691708e-3 -
                th *( 9.27062484e-6 +
                     th2 * 1.78343643e-10
                    ) +
                sqrts *( 4.76534122e-6 +
                        th2 * 1.63410736e-9
                       )
               )
           )
    
#     anum_th =   (      7.352128400 + 
#                  th *(-1.091856422e-01 + 
#                  th *  1.195430112e-03) -
#                   s *  7.232688130e-03)

#     aden_th =   (      7.286067390e-03 +
#                  th *(-9.216710840e-05 + 
#                  th *( 1.105171719e-06 + 
#                  th *  7.232367440e-10)) +
#                   s *(-9.270624840e-06 - 
#                 th2 *  5.350309290e-10 + 
#           th *sqrts *  3.268214720e-09))

#     anum_s =    (      2.96938239 -
#                  th *  7.23268813e-03 +
#                   s *  4.24764682e-03)

#     aden_s =    (      2.14691708e-03 -
#                  th *( 9.27062484e-06 +
#                 th2 *  1.78343643e-10) +
#               sqrts *( 7.14801183e-06 + 
#                 th2 *  2.45116104e-09))

    anum_p = ( 1.04004591e-2 +
              th2 * 1.03970529e-7 +
              s * 5.18761880e-6
             )
    aden_p = 5.30848875e-6 * xr.ones_like(s)

#     s = s.where(p!=0.)
#     th = th.where(p!=0.) 
#     th2 = th2.where(p!=0.) 
#     p = p.where(p!=0.)
#     anum = anum.where(p!=0.)
#     aden = aden.where(p!=0.)
#     anum_p = anum_p.where(p!=0.)
#     aden_p = aden_p.where(p!=0.)

    pth = p*th
    anum = anum + p *( 1.04004591e-2 +
                      th2 *  1.03970529e-7 +
                      s *  5.18761880e-6 -
                      p *( 3.24041825e-8 +
                          th2 *  1.23869360e-11
                         )
                     )
    aden = aden + p *( 5.30848875e-6 -
                      pth *(th2 *  3.03175128e-16 +
                            p *  1.27934137e-17
                           )
                     )

#     anum_s = anum_s +                    p *  5.1876188e-06

#     anum_th = anum_th +                pth *( 2.07941058e-07 - 
#                                          p *  2.47738720e-11)

#     aden_th = aden_th -         p *p *(th2 *  9.09525384e-16 +
#                                          p *  1.27934137e-17) 

    anum_p = anum_p - p *( 6.4808365e-8 + th2 * 2.4773872e-11)

    aden_p = aden_p - pth *(th2 * 6.06350256e-16 +
                            p * 3.83802411e-17)


    rec_aden = aden**-1

    rho = anum*rec_aden
#     rho_s = (anum_s - aden_s*rho)*rec_aden
#     rho_th = (anum_th - aden_th*rho)*rec_aden
    rho_p = (anum_p - aden_p*rho)*rec_aden
    
    return np.sqrt(rho_p*1e-4)**-1

In [15]:
# for itime in range(1101600+dnf,1259280+dnf,dnf):
# for itime in range(1043280+dnf,1101600+dnf,dnf):
for itime in range(943920,943920+dnf,dnf):
    
    dsz = xr.open_dataset(op.join(savedir,'%4d/Zs_membs_parallel-%2d_%010d.nc' 
                                  % (year,nremap,itime)),
                          chunks={'nmemb':membchunk,'XC':xchunk,'XG':xchunk,
                                  'YC':ychunk,'YG':ychunk}
                         )
        
    grids = Grid(dsz, periodic=['X'], 
                 coords={'Z':{'center':'sig','outer':'sigp1'},
                         'Y':{'center':'YC','left':'YG'},
                         'X':{'center':'XC','left':'XG'}}
                )
    
    for nmemb in range(24,36):
        if nmemb == 24:
            ds = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                         % (nmemb,year)), 
                                grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                prefix=['diag_ocnSnap',
                                        'diag_Tbgt2D','diag_Tbgt3D',
                                        'diag_Sbgt2D','diag_Sbgt3D'], 
                                delta_t=2e2, iters=itime, 
                                chunks={'XC':xchunk,'XG':xchunk}
                                ).isel(time=-1)
        else:
            ds = xr.concat([ds, open_mdsdataset(op.join(ddir,
                                                        'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                                        % (nmemb,year)), 
                                                grid_dir=op.join(ddir,
                                                                 'grid_chaO/gridMIT_update1/'),
                                                prefix=['diag_ocnSnap',
                                                        'diag_Tbgt2D','diag_Tbgt3D',
                                                        'diag_Sbgt2D','diag_Sbgt3D'], 
                                                delta_t=2e2, iters=itime, 
                                                chunks={'XC':xchunk,'XG':xchunk}
                                               ).isel(time=-1)],
                            dim='nmemb')
    ds.coords['nmemb'] = ('nmemb',np.arange(0,12,1))
    
#     cs = xr.apply_ufunc(sound_speed, 
#                         ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
#                         ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
#                         -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
#                         dask='parallelized', output_dtypes=[float,]
#                        ).where(ds.maskC!=0.).compute().chunk({'nmemb':membchunk,
#                                                               'XC':xchunk,'YC':ychunk})
    cs = sound_speed(ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                     ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                     -grav*ds.Z*rhoConst*1e-4
                    ).where(ds.maskC!=0.)
    rho_s = xr.apply_ufunc(drhods, 
                           ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                           ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
                           -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                           dask='parallelized', output_dtypes=[float,]
                          ).where(ds.maskC!=0.)
    rho_th = xr.apply_ufunc(drhodt, 
                            ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                            ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
                            -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                            dask='parallelized', output_dtypes=[float,]
                           ).where(ds.maskC!=0.)
            
    fine_drf = np.squeeze(ds.drF.sel(Z=slice(None,None)))
    fine_drc = np.squeeze(ds.drC.sel(Zp1=slice(None,None)))
#     drf_finer,mapindex,mapfact,cellindex = finegrid.finegrid(fine_drf, fine_drc, nremap)
    
    grid = Grid(ds, periodic=['X'])
    
#################################
#     rho_insitu = xr.apply_ufunc(jmd95.densjmd95, 
#                                 ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
#                                 ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
# #                                 (ds.PHIHYD.sel(YC=slice(ySstart-12**-1,yend+2*12**-1))
#                                 -grav*ds.Z*rhoConst*1e-4, # pressure in [dbar]!!!!
#                                 dask='parallelized', output_dtypes=[float,]
#                                ).where(ds.maskC!=0.).compute().chunk({'nmemb':membchunk,
#                                                                       'XC':xchunk,'YC':ychunk}
#                                                                     )
    
#     rhox = xr.apply_ufunc(jmd95.densjmd95, 
#                           grid.interp(ds.SALT,'X').sel(YC=slice(yNstart-12**-1,
#                                                                 yend+2*12**-1)), 
#                           grid.interp(ds.THETA,'X').sel(YC=slice(yNstart-12**-1,
#                                                                  yend+2*12**-1)), 
# #                           (grid.interp(ds.PHIHYD,'X').sel(YC=slice(ySstart-12**-1,
# #                                                                    yend+2*12**-1))
#                           -grav*ds.Z*rhoConst*1e-4, 
#                           dask='parallelized', output_dtypes=[float,]
#                          ).where(ds.maskW!=0.).compute().chunk({'nmemb':membchunk,
#                                                                 'XG':xchunk,'YC':ychunk})
    
#     rhoy = xr.apply_ufunc(jmd95.densjmd95, 
#                           grid.interp(ds.SALT,'Y',boundary='fill'
#                                      ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
#                           grid.interp(ds.THETA,'Y',boundary='fill'
#                                      ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
# #                           (grid.interp(ds.PHIHYD,'Y',boundary='fill'
# #                                       ).sel(YG=slice(ySstart-12**-1,yend+2*12**-1))
#                           -grav*ds.Z*rhoConst*1e-4,
#                           dask='parallelized', output_dtypes=[float,]
#                          ).where(ds.maskS!=0.).compute().chunk({'nmemb':membchunk,
#                                                                 'XC':xchunk,'YG':ychunk})

    rho_insitu = xr.apply_ufunc(densjmd95, 
                                ds.SALT.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                                ds.THETA.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)),
                                -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                                dask='parallelized', output_dtypes=[float,]
                               ).where(ds.maskC!=0.).compute().chunk({'nmemb':membchunk,
                                                                      'XC':xchunk,'YC':ychunk})
    rhox = xr.apply_ufunc(densjmd95, 
                          grid.interp(ds.SALT,'X').sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                          grid.interp(ds.THETA,'X').sel(YC=slice(yNstart-12**-1,yend+2*12**-1)), 
                          -grav*ds.Z*rhoConst*1e-4, 
                          dask='parallelized', output_dtypes=[float,]
                         ).where(ds.maskW!=0.).compute().chunk({'nmemb':membchunk,
                                                                'XG':xchunk,'YC':ychunk})
    rhoy = xr.apply_ufunc(densjmd95, 
                          grid.interp(ds.SALT,'Y',boundary='fill'
                                     ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          grid.interp(ds.THETA,'Y',boundary='fill'
                                     ).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)), 
                          -grav*ds.Z*rhoConst*1e-4,
                          dask='parallelized', output_dtypes=[float,]
                         ).where(ds.maskS!=0.).compute().chunk({'nmemb':membchunk,
                                                                'XC':xchunk,'YG':ychunk})
    
########################################
# #     dpdx = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.) * grid.interp(ds.dyG,'X'),
# #                       'X'
# #                      ) * ds.rAw**-1
# #            ).where(ds.maskW!=0.)
# #     dpdy = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.) 
# #                       * grid.interp(ds.dxG,'Y',boundary='fill'),
# #                       'Y',boundary='fill'
# #                      ) * ds.rAs**-1
# #            ).where(ds.maskS!=0.)
    dpdx = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.),'X') 
            * ds.dxC**-1
           ).where(ds.maskW!=0.)
    dpdy = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.),'Y',boundary='fill') 
            * ds.dyC**-1
           ).where(ds.maskS!=0.)
# #     dpdx.coords['nmemb'] = ('nmemb',ds.nmemb.data)
# #     dpdy.coords['nmemb'] = ('nmemb',ds.nmemb.data)
    
########################################
    swfrac = .62 * np.exp(ds.Zl/.6) + (1-.62) * np.exp(ds.Zl/20.)
    swfrac1 = (.62 * np.exp(ds.Zl.shift(Zl=-1)/.6) + (1-.62) 
               * np.exp(ds.Zl.shift(Zl=-1)/20.))
    Qsw = (ds.oceQsw / (rhoConst*Cp) / (ds.drF*ds.hFacC) 
           * (swfrac - swfrac1).data
          ).where(ds.maskC!=0.)
    Tflx = ((ds.TFLUX - ds.oceQsw) 
            / (rhoConst*Cp*ds.drF[0]*ds.hFacC.isel(Z=0))
           ).where(ds.maskInC!=0.)
    tsurf_corr = 0.                  # linFSConverveTr = F.
    Surf_corr = (tsurf_corr - ds.WTHMASS[:,0].where(ds.maskInC!=0.)
                ) / (ds.drF[0]*ds.hFacC.isel(Z=0))
#     DibaT = (ds.Diss_TH.where(ds.maskC!=0.) + Qsw).load()
#     DibaT.isel(Z=0)[:] += (Tflx + Surf_corr).load()
    DibaT = ds.Diss_TH.where(ds.maskC!=0.) + Qsw
    DibaT = DibaT + xr.concat([xr.DataArray((Tflx+Surf_corr).data.reshape((len(ds.nmemb),1,
                                                                           len(ds.YC),len(ds.XC)
                                                                          )),
                                            dims=['nmemb','Z','YC','XC'],
                                            coords={'nmemb':ds.nmemb.data,
                                                    'Z':np.array([ds.Z[0].data]),
                                                    'YC':ds.YC.data,'XC':ds.XC.data}),
                               xr.zeros_like(ds.Diss_TH.isel(Z=slice(1,None))
                                            ).reset_coords(drop=True)],
                              dim='Z')

########################################
    Sflx = (ds.SFLUX 
            / (rhoConst*ds.drF[0]*ds.hFacC.isel(Z=0))
           ).where(ds.maskInC!=0.)
    ssurf_corr = 0.                  # linFSConverveTr = F.
    Surf_corr = (ssurf_corr - ds.WSLTMASS[:,0].where(ds.maskInC!=0.)
                ) / (ds.drF[0]*ds.hFacC.isel(Z=0))
#     DibaS = ds.Diss_SLT.where(ds.maskC!=0.).load()
#     DibaS.isel(Z=0)[:] += (Sflx + Surf_corr).load()
    DibaS = ds.Diss_SLT.where(ds.maskC!=0.)
    DibaS = DibaS + xr.concat([xr.DataArray((Sflx+Surf_corr).data.reshape((len(ds.nmemb),1,
                                                                           len(ds.YC),len(ds.XC)
                                                                          )),
                                            dims=['nmemb','Z','YC','XC'],
                                            coords={'nmemb':ds.nmemb.data,
                                                    'Z':np.array([ds.Z[0].data]),
                                                    'YC':ds.YC.data,'XC':ds.XC.data}),
                               xr.zeros_like(ds.Diss_SLT.isel(Z=slice(1,None))
                                            ).reset_coords(drop=True)],
                              dim='Z')
    
    comp = grid.interp(ds.WVEL,'Z',boundary='fill').chunk({'Z':-1}) * grav**2 * cs**-2
    
##########################################
    start = time.time()    
# #     diaTdz = xr.DataArray(np.ones((len(ds.nmemb),
# #                                    len(ds.YC.sel(YC=slice(yNstart-12**-1,
# #                                                           yend+2*12**-1))),
# #                                    len(ds.XC),len(siglayers))),
# #                           dims=['nmemb','YC','XC','sig'],
# #                           coords={'nmemb':range(len(ds.nmemb),),
# #                                   'YC':ds.YC.sel(YC=slice(yNstart-12**-1,
# #                                                           yend+2*12**-1)).data,
# #                                   'XC':ds.XC.data,'sig':siglayers}
# #                          ) * np.nan
# #     diaSdz = diaTdz.copy()
# #     sdz = diaTdz.copy()
# #     tdz = diaTdz.copy()
# #     pxdz = xr.DataArray(np.ones((len(ds.nmemb),
# #                                  len(ds.YC.sel(YC=slice(yNstart-12**-1,yend+2*12**-1))),
# #                                  len(ds.XG),len(siglayers))),
# #                         dims=['nmemb','YC','XG','sig'],
# #                         coords={'nmemb':range(len(ds.nmemb),),
# #                                 'YC':ds.YC.sel(YC=slice(yNstart-12**-1,
# #                                                         yend+2*12**-1)).data,
# #                                 'XG':ds.XG.data,'sig':siglayers}
# #                        ) * np.nan
# #     pydz = xr.DataArray(np.ones((len(ds.nmemb),
# #                                  len(ds.YG.sel(YG=slice(yNstart-12**-1,yend+2*12**-1))),
# #                                  len(ds.XC),len(siglayers))),
# #                         dims=['nmemb','YG','XC','sig'],
# #                         coords={'nmemb':range(len(ds.nmemb),),
# #                                 'YG':ds.YG.sel(YG=slice(yNstart-12**-1,
# #                                                         yend+2*12**-1)).data,
# #                                 'XC':ds.XC.data,'sig':siglayers}
# #                        ) * np.nan
# #     udz = pxdz.copy()
# #     vdz = pydz.copy()
    
# #     for tt in range(len(ds.nmemb)):
# #         tmp1 = xr.apply_ufunc(layers_numpy, 
# #                               dpdx.isel(nmemb=tt).sel(YC=slice(yNstart-12**-1,
# #                                                                yend+2*12**-1)
# #                                                      ).chunk({'XG':xchunk}),
# #                               (rhox.isel(nmemb=tt)-rhoConst),
# #                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
# #                                       'mapindex':mapindex,'cellindex':cellindex,
# #                                       'drf_finer':drf_finer},
# #                               dask='parallelized',
# #                               input_core_dims=[['Z'],['Z']], 
# #                               output_core_dims=[['sig']],
# #                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
# #                              ).compute()

# #         tmp2 = xr.apply_ufunc(layers_numpy, 
# #                               dpdy.isel(nmemb=tt).sel(YG=slice(yNstart-12**-1,
# #                                                                yend+2*12**-1)
# #                                                      ).chunk({'YG':ychunk}),
# #                               (rhoy.isel(nmemb=tt)-rhoConst),
# #                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
# #                                       'mapindex':mapindex,'cellindex':cellindex,
# #                                       'drf_finer':drf_finer},
# #                               dask='parallelized',
# #                               input_core_dims=[['Z'],['Z']], 
# #                               output_core_dims=[['sig']],
# #                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
# #                              ).compute()

# #         pxdz[tt] = tmp1.data
# #         pydz[tt] = tmp2.data
# #         del tmp1, tmp2
    pxdz = layers_apply(dpdx.sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
                                ).chunk({'nmemb':membchunk,'XG':xchunk}),
                        (rhox-rhoConst),
                        siglayers, fine_drf, fine_drc, fine_fact=nremap,
                        lev_name='Z', Tlev_name='sig').compute()
    pydz = layers_apply(dpdy.sel(YG=slice(yNstart-12**-1,yend+2*12**-1)
                                ).chunk({'nmemb':membchunk,'YG':ychunk}),
                        (rhoy-rhoConst),
                        siglayers, fine_drf, fine_drc, fine_fact=nremap,
                        lev_name='Z', Tlev_name='sig').compute()
        
#         tmp3 = xr.apply_ufunc(layers_numpy, 
#                               DibaT.isel(nmemb=tt).where(ds.maskC!=0.
#                                                         ).sel(YC=slice(yNstart-12**-1,
#                                                                        yend+2*12**-1)
#                                                              ).chunk({'Z':-1,'YC':ychunk,
#                                                                       'XC':xchunk}),
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized',
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tmp4 = xr.apply_ufunc(layers_numpy, 
#                               DibaS.isel(nmemb=tt).where(ds.maskC!=0.
#                                                         ).sel(YC=slice(yNstart-12**-1,
#                                                                        yend+2*12**-1)
#                                                              ).chunk({'Z':-1,'YC':ychunk,
#                                                                       'XC':xchunk}),
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized',
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         diaTdz[tt] = tmp3.data
#         diaSdz[tt] = tmp4.data
#         del tmp3, tmp4
    diaTdz = layers_apply((rho_th*DibaT).where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,
                                                                          yend+2*12**-1)
                                                                ).chunk({'nmemb':membchunk,'Z':-1,
                                                                         'YC':ychunk,'XC':xchunk}),
                          (rho_insitu-rhoConst),
                          siglayers, fine_drf, fine_drc, fine_fact=nremap,
                          lev_name='Z', Tlev_name='sig').compute()
    diaSdz = layers_apply((rho_s*DibaS).where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,
                                                                         yend+2*12**-1)
                                                               ).chunk({'nmemb':membchunk,'Z':-1,
                                                                        'YC':ychunk,'XC':xchunk}),
                          (rho_insitu-rhoConst),
                          siglayers, fine_drf, fine_drc, fine_fact=nremap,
                          lev_name='Z', Tlev_name='sig').compute()
#     drdTdz = layers_apply(rho_th.where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
#                                                         ).chunk({'nmemb':membchunk,
#                                                                  'YC':ychunk,'XC':xchunk}),
#                           (rho_insitu-rhoConst),
#                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
#                           lev_name='Z', Tlev_name='sig').compute()
#     drdsdz = layers_apply(rho_s.where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
#                                                        ).chunk({'nmemb':membchunk,
#                                                                 'YC':ychunk,'XC':xchunk}),
#                           (rho_insitu-rhoConst),
#                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
#                           lev_name='Z', Tlev_name='sig').compute()
    cmpdz = layers_apply(comp.where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
                                                     ).chunk({'nmemb':membchunk,
                                                              'YC':ychunk,'XC':xchunk}),
                         (rho_insitu-rhoConst),
                         siglayers, fine_drf, fine_drc, fine_fact=nremap,
                         lev_name='Z', Tlev_name='sig').compute()
        
#         tmp5 = xr.apply_ufunc(layers_numpy, 
#                               ds.THETA.isel(nmemb=tt).where(ds.maskC!=0.
#                                                            ).sel(YC=slice(yNstart-12**-1,
#                                                                           yend+2*12**-1)
#                                                                 ).chunk({'YC':ychunk,
#                                                                          'XC':xchunk}),
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized',
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tmp6 = xr.apply_ufunc(layers_numpy, 
#                               ds.SALT.isel(nmemb=tt).where(ds.maskC!=0.
#                                                           ).sel(YC=slice(yNstart-12**-1,
#                                                                          yend+2*12**-1)
#                                                                ).chunk({'YC':ychunk,
#                                                                         'XC':xchunk}),
#                               (rho_insitu.isel(nmemb=tt)-rhoConst),
#                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
#                                       'mapindex':mapindex,'cellindex':cellindex,
#                                       'drf_finer':drf_finer},
#                               dask='parallelized',
#                               input_core_dims=[['Z'],['Z']], 
#                               output_core_dims=[['sig']],
#                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
#                              ).compute()

#         tdz[tt] = tmp5.data
#         sdz[tt] = tmp6.data
#         del tmp5, tmp6
    tdz = layers_apply(ds.THETA.where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
                                                       ).chunk({'nmemb':membchunk,
                                                                'YC':ychunk,'XC':xchunk}),
                       (rho_insitu-rhoConst),
                       siglayers, fine_drf, fine_drc, fine_fact=nremap,
                       lev_name='Z', Tlev_name='sig').compute()
    sdz = layers_apply(ds.SALT.where(ds.maskC!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
                                                      ).chunk({'nmemb':membchunk,
                                                               'YC':ychunk,'XC':xchunk}),
                       (rho_insitu-rhoConst),
                       siglayers, fine_drf, fine_drc, fine_fact=nremap,
                       lev_name='Z', Tlev_name='sig').compute()
        
# #         tmp7 = xr.apply_ufunc(layers_numpy, 
# #                               ds.UVEL.isel(nmemb=tt).where(ds.maskW!=0.
# #                                                           ).sel(YC=slice(yNstart-12**-1,
# #                                                                          yend+2*12**-1)
# #                                                                ).chunk({'YC':ychunk,
# #                                                                         'XG':xchunk}),
# #                               (rhox.isel(nmemb=tt)-rhoConst),
# #                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
# #                                       'mapindex':mapindex,'cellindex':cellindex,
# #                                       'drf_finer':drf_finer},
# #                               dask='parallelized',
# #                               input_core_dims=[['Z'],['Z']], 
# #                               output_core_dims=[['sig']],
# #                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
# #                              ).compute()

# #         tmp8 = xr.apply_ufunc(layers_numpy, 
# #                               ds.VVEL.isel(nmemb=tt).where(ds.maskS!=0.
# #                                                           ).sel(YG=slice(yNstart-12**-1,
# #                                                                          yend+2*12**-1)
# #                                                                ).chunk({'YG':ychunk,
# #                                                                         'XC':xchunk}),
# #                               (rhoy.isel(nmemb=tt)-rhoConst),
# #                               kwargs={'thetalayers':siglayers,'mapfact':mapfact,
# #                                       'mapindex':mapindex,'cellindex':cellindex,
# #                                       'drf_finer':drf_finer},
# #                               dask='parallelized',
# #                               input_core_dims=[['Z'],['Z']], 
# #                               output_core_dims=[['sig']],
# #                               output_dtypes=[float,], output_sizes={'sig':siglayers.size}
# #                              ).compute()

# #         udz[tt] = tmp7.data
# #         vdz[tt] = tmp8.data
# #         del tmp7, tmp8
    udz = layers_apply(ds.UVEL.where(ds.maskW!=0.).sel(YC=slice(yNstart-12**-1,yend+2*12**-1)
                                                      ).chunk({'nmemb':membchunk,
                                                               'YC':ychunk,'XG':xchunk}),
                       (rhox-rhoConst),
                       siglayers, fine_drf, fine_drc, fine_fact=nremap,
                       lev_name='Z', Tlev_name='sig').compute()
    vdz = layers_apply(ds.VVEL.where(ds.maskS!=0.).sel(YG=slice(yNstart-12**-1,yend+2*12**-1)
                                                      ).chunk({'nmemb':membchunk,
                                                               'YG':ychunk,'XC':xchunk}),
                       (rhoy-rhoConst),
                       siglayers, fine_drf, fine_drc, fine_fact=nremap,
                       lev_name='Z', Tlev_name='sig').compute()
    

#     pxdz = pxdz.chunk({'nmemb':membchunk,'sig':sigchunk,'XG':xchunk,'YC':ychunk})
#     pydz = pydz.chunk({'nmemb':membchunk,'sig':sigchunk,'XC':xchunk,'YG':ychunk})
    diaTdz = diaTdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
    diaSdz = diaSdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#     drdTdz = drdTdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#     drdsdz = drdsdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
    cmpdz = cmpdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#     tdz = tdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#     sdz = sdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#     udz = udz.chunk({'nmemb':membchunk,'sig':sigchunk,'XG':xchunk,'YC':ychunk})
#     vdz = vdz.chunk({'nmemb':membchunk,'sig':sigchunk,'XC':xchunk,'YG':ychunk})
    
    del rho_insitu
    del rhoy, rhox
    end = time.time()
    print("Interpolation done. Lapse:", end-start)
    
    
##########################################    
    buoyb = -grav * dsz.sig * rhoConst**-1
    buoybp1 = -grav * dsz.sigp1 * rhoConst**-1
    
#     _dT = -grids.diff(tdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#                       * dsz.dzetaF**-1,
#                       'Z',boundary='fill')**-1
#     _ds = -grids.diff(sdz.chunk({'nmemb':membchunk,'YC':ychunk,'XC':xchunk})
#                       * dsz.dzetaF**-1,
#                       'Z',boundary='fill')**-1
# #     varpidz = (grids.interp(dbdT,'Z',boundary='fill') * diaTdz
# #                + grids.interp(dbds,'Z',boundary='fill') * diaSdz
# #               ).compute()
#     varpidz_db = (grids.interp(_dT,'Z',boundary='fill') * diaTdz
#                   + grids.interp(_ds,'Z',boundary='fill') * diaSdz
#                  )
#     varpidz = (varpidz_db * -grids.diff(buoybp1,'Z',boundary='fill')
#                + cmpdz).compute()
    varpidzdr = diaTdz + diaSdz
    varpidz = (-varpidzdr * grav*rhoConst**-1 
               + cmpdz
              ).compute()
    
    dsave = varpidz.to_dataset(name='varpidz')
    dsave.coords['sigp1'] = ('sigp1',sigp1layers)
    dsave.to_netcdf(op.join(savedir,'%4d/Varpidz-compressible_membs_parallel-%2d_%010d.nc' 
                            % (year,nremap,itime)))
    dsave.close()
#     dsave = varpidz_db.to_dataset(name='varpidz_db')
    dsave = vdz.to_dataset(name='vdz')
    dsave['udz'] = udz
    dsave['tdz'] = tdz
    dsave['sdz'] = sdz
    dsave['pxdz'] = pxdz
    dsave['pydz'] = pydz
    dsave.coords['sigp1'] = ('sigp1',sigp1layers)
    dsave.to_netcdf(op.join(savedir,'%4d/Dyn_membs_parallel-%2d_%010d.nc' 
                            % (year,nremap,itime)))
    dsave.close()
    dsz.close()
    ds.close()

#     del cs, rho_th, rho_s
    del varpidz, tdz, sdz, diaTdz, diaSdz, cmpdz
    del vdz, udz, pydz, pxdz
#     client.restart()
    
    print(itime)

Interpolation done. Lapse: 1344.8100290298462
943920


In [ ]:
for year in range(1966,1968):
    if year == 1963:
        ntimes = np.arange( 943920    , 943920+dnf,dnf,dtype=int)
        ystart = ySstart
    else:
        ystart = yNstart
        if year == 1964:
            ntimes = np.arange( 943920+dnf,1101600+dnf,dnf,dtype=int)
        elif year == 1965:
            ntimes = np.arange(1101600+dnf,1259280+dnf,dnf,dtype=int)
        elif year == 1966:
            ntimes = np.arange(1358640+dnf,1416960+dnf,dnf,dtype=int)
        else:
            ntimes = np.arange(1416960+dnf,1574640+dnf,dnf,dtype=int)
        
    for itime in ntimes:
        for nmemb in range(24,36):
            if nmemb == 24:
                ds = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                             % (nmemb,year)), 
                                    grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                    prefix=['diag_ocnSnap',
                                            'diag_Tbgt2D','diag_Tbgt3D',
                                            'diag_Sbgt2D','diag_Sbgt3D'], 
                                    delta_t=2e2, iters=itime, 
                                    chunks={'XC':xchunk,'XG':xchunk}
                                    ).isel(time=-1)
            else:
                ds = xr.concat([ds, open_mdsdataset(op.join(ddir,
                                                            'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                                            % (nmemb,year)), 
                                                    grid_dir=op.join(ddir,
                                                                     'grid_chaO/gridMIT_update1/'),
                                                    prefix=['diag_ocnSnap',
                                                            'diag_Tbgt2D','diag_Tbgt3D',
                                                            'diag_Sbgt2D','diag_Sbgt3D'], 
                                                    delta_t=2e2, iters=itime, 
                                                    chunks={'XC':xchunk,'XG':xchunk}
                                                   ).isel(time=-1)],
                                dim='nmemb')
        ds.coords['nmemb'] = ('nmemb',range(12))


        dHT0 = xr.open_dataset(op.join(savedir,'IC_temperature-bias_%2dS-%2dN.nc'
                                       % (np.abs(ySstart),yend))).dHT0
        dHS0 = xr.open_dataset(op.join(savedir,'IC_salinity-bias_%2dS-%2dN.nc'
                                       % (np.abs(ySstart),yend))).dHS0
        theta = ds.THETA - dHT0
        salt = ds.SALT - dHS0

        cs = sound_speed(salt.sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                         theta.sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                         -grav*ds.Z*rhoConst*1e-4
                        ).where(ds.maskC!=0.)
        rho_s = xr.apply_ufunc(drhods, 
                               salt.sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                               theta.sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                               -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                               dask='parallelized', output_dtypes=[float,]
                              ).where(ds.maskC!=0.)
        rho_th = xr.apply_ufunc(drhodt, 
                                salt.sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                                theta.sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                                -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                                dask='parallelized', output_dtypes=[float,]
                               ).where(ds.maskC!=0.)

        fine_drf = np.squeeze(ds.drF.sel(Z=slice(None,None)))
        fine_drc = np.squeeze(ds.drC.sel(Zp1=slice(None,None)))

        grid = Grid(ds, periodic=['X'])

########################################
#                 Rhos                 #
########################################
        rho_insitu = xr.apply_ufunc(densjmd95, 
                                    salt.sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                                    theta.sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                                    -grav*ds.Z*rhoConst*1e-4,             # pressure in [dbar]!!!!
                                    dask='parallelized', output_dtypes=[float,]
                                   ).where(ds.maskC!=0.).compute().chunk({'nmemb':membchunk,
                                                                          'XC':xchunk,'YC':ychunk})
        rhox = xr.apply_ufunc(densjmd95, 
                              grid.interp(salt,'X').sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                              grid.interp(theta,'X').sel(YC=slice(ystart-12**-1,yend+2*12**-1)), 
                              -grav*ds.Z*rhoConst*1e-4, 
                              dask='parallelized', output_dtypes=[float,]
                             ).where(ds.maskW!=0.).compute().chunk({'nmemb':membchunk,
                                                                    'XG':xchunk,'YC':ychunk})
        rhoy = xr.apply_ufunc(densjmd95, 
                              grid.interp(salt,'Y',boundary='fill'
                                         ).sel(YG=slice(ystart-12**-1,yend+2*12**-1)), 
                              grid.interp(theta,'Y',boundary='fill'
                                         ).sel(YG=slice(ystart-12**-1,yend+2*12**-1)), 
                              -grav*ds.Z*rhoConst*1e-4,
                              dask='parallelized', output_dtypes=[float,]
                             ).where(ds.maskS!=0.).compute().chunk({'nmemb':membchunk,
                                                                    'XC':xchunk,'YG':ychunk})
        rhoz = xr.apply_ufunc(densjmd95, 
                              grid.interp(grid.interp(salt,'Y',boundary='fill'),
                                          'X').sel(YG=slice(ystart-12**-1,yend+2*12**-1)), 
                              grid.interp(grid.interp(theta,'Y',boundary='fill'),
                                          'X').sel(YG=slice(ystart-12**-1,yend+2*12**-1)), 
                              -grav*ds.Z*rhoConst*1e-4,
                              dask='parallelized', output_dtypes=[float,]
                             ).where(grid.interp(ds.maskS,'X')!=0.
                                    ).compute().chunk({'nmemb':membchunk,
                                                       'XG':xchunk,'YG':ychunk})
#         print(rho_insitu)

##########################################
#                   Zs                   #
##########################################
        start = time.time()
        zz, _ = xr.broadcast(ds.Z, rho_insitu.isel(Z=0).sel(YC=slice(ystart-12**-1,
                                                                     yend+2*12**-1))
                            )
        zzy, _ = xr.broadcast(ds.Z, rhoy.isel(Z=0).sel(YG=slice(ystart-12**-1,
                                                                yend+2*12**-1))
                             )
        zzx, _ = xr.broadcast(ds.Z, rhox.isel(Z=0).sel(YC=slice(ystart-12**-1,
                                                                yend+2*12**-1))
                             )
        zz = zz.chunk({'nmemb':membchunk,'YC':ychunk,
                       'XC':xchunk}).transpose('nmemb','Z','YC','XC')
        zzy = zzy.chunk({'nmemb':membchunk,'YG':ychunk,
                         'XC':xchunk}).transpose('nmemb','Z','YG','XC')
        zzx = zzx.chunk({'nmemb':membchunk,'YC':ychunk,
                         'XG':xchunk}).transpose('nmemb','Z','YC','XG')

        dzetaF = layers_apply(xr.ones_like(zz).where(ds.maskC!=0.).chunk({'nmemb':membchunk,
                                                                          'XC':xchunk,
                                                                          'YC':ychunk}),
                              (rho_insitu-rhoConst),
                              siglayers, fine_drf, fine_drc, fine_fact=nremap,
                              lev_name='Z', Tlev_name='sig').compute()
        zdz = layers_apply(zz, (rho_insitu-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()

        dzetaFx = layers_apply(xr.ones_like(zzx).where(ds.maskW!=0.).chunk({'nmemb':membchunk,
                                                                            'XG':xchunk,
                                                                            'YC':ychunk}),
                               (rhox-rhoConst),
                               siglayers, fine_drf, fine_drc, fine_fact=nremap,
                               lev_name='Z', Tlev_name='sig').compute()
        zxdz = layers_apply(zzx, (rhox-rhoConst),
                            siglayers, fine_drf, fine_drc, fine_fact=nremap,
                            lev_name='Z', Tlev_name='sig').compute()

        dzetaFy = layers_apply(xr.ones_like(zzy).where(ds.maskS!=0.).chunk({'nmemb':membchunk,
                                                                            'XC':xchunk,
                                                                            'YG':ychunk}),
                               (rhoy-rhoConst),
                               siglayers, fine_drf, fine_drc, fine_fact=nremap,
                               lev_name='Z', Tlev_name='sig').compute()
        zydz = layers_apply(zzy, (rhoy-rhoConst),
                            siglayers, fine_drf, fine_drc, fine_fact=nremap,
                            lev_name='Z', Tlev_name='sig').compute()

        dzetaFz = layers_apply(xr.ones_like(rhoz).where(grid.interp(ds.maskS,'X')!=0.
                                                       ).chunk({'nmemb':membchunk,'XG':xchunk,
                                                                'YG':ychunk}),
                               (rhoz-rhoConst),
                               siglayers, fine_drf, fine_drc, fine_fact=nremap,
                               lev_name='Z', Tlev_name='sig').compute()

        del zz, zzy, zzx
        end = time.time()
        print("Zs interpolated. Lapse:", end-start)

    ##########################################
        dsave = zdz.to_dataset(name='zdz')
        dsave['zydz'] = zydz
        dsave['zxdz'] = zxdz
        dsave['dzetaF'] = dzetaF
        dsave['dzetaFx'] = dzetaFx
        dsave['dzetaFy'] = dzetaFy
        dsave['dzetaFz'] = dzetaFz
        dsave.coords['sigp1'] = ('sigp1',sigp1layers)
#         dsave.to_netcdf(op.join(savedir,'%4d/Zs_membs_parallel-%2d_%010d.nc' 
#                                 % (year,nremap,itime)))
        dsave.to_zarr(op.join(savedir,'%4d/Zs/%07d/' % (year,itime)), mode='w')
        dsave.close()

        del zdz, zydz, zxdz, dzetaF, dzetaFz, dzetaFy, dzetaFx    
        print("Zs saved.")

##########################################
#          Dynamical variables           #
##########################################
        start = time.time()

        swfrac = .62 * np.exp(ds.Zl/.6) + (1-.62) * np.exp(ds.Zl/20.)
        swfrac1 = (.62 * np.exp(ds.Zl.shift(Zl=-1)/.6) + (1-.62) 
                   * np.exp(ds.Zl.shift(Zl=-1)/20.))
        Qsw = (ds.oceQsw / (rhoConst*Cp) / (ds.drF*ds.hFacC) 
               * (swfrac - swfrac1).data
              ).where(ds.maskC!=0.)
        Tflx = ((ds.TFLUX - ds.oceQsw) 
                / (rhoConst*Cp*ds.drF[0]*ds.hFacC.isel(Z=0))
               ).where(ds.maskInC!=0.)
        tsurf_corr = 0.                  # linFSConverveTr = F.
        Surf_corr = (tsurf_corr - ds.WTHMASS[:,0].where(ds.maskInC!=0.)
                    ) / (ds.drF[0]*ds.hFacC.isel(Z=0))
        DibaT = ds.Diss_TH.where(ds.maskC!=0.) + Qsw
        DibaT = DibaT + xr.concat([xr.DataArray((Tflx+Surf_corr).data.reshape((len(ds.nmemb),1,
                                                                               len(ds.YC),len(ds.XC)
                                                                              )),
                                                dims=['nmemb','Z','YC','XC'],
                                                coords={'nmemb':ds.nmemb.data,
                                                        'Z':np.array([ds.Z[0].data]),
                                                        'YC':ds.YC.data,'XC':ds.XC.data}),
                                   xr.zeros_like(ds.Diss_TH.isel(Z=slice(1,None))
                                                ).reset_coords(drop=True)],
                                  dim='Z')
##########################################
        Sflx = (ds.SFLUX 
                / (rhoConst*ds.drF[0]*ds.hFacC.isel(Z=0))
               ).where(ds.maskInC!=0.)
        ssurf_corr = 0.                  # linFSConverveTr = F.
        Surf_corr = (ssurf_corr - ds.WSLTMASS[:,0].where(ds.maskInC!=0.)
                    ) / (ds.drF[0]*ds.hFacC.isel(Z=0))
        DibaS = ds.Diss_SLT.where(ds.maskC!=0.)
        DibaS = DibaS + xr.concat([xr.DataArray((Sflx+Surf_corr).data.reshape((len(ds.nmemb),1,
                                                                               len(ds.YC),len(ds.XC)
                                                                              )),
                                                dims=['nmemb','Z','YC','XC'],
                                                coords={'nmemb':ds.nmemb.data,
                                                        'Z':np.array([ds.Z[0].data]),
                                                        'YC':ds.YC.data,'XC':ds.XC.data}),
                                   xr.zeros_like(ds.Diss_SLT.isel(Z=slice(1,None))
                                                ).reset_coords(drop=True)],
                                  dim='Z')
        comp = grid.interp(ds.WVEL,'Z',boundary='fill').chunk({'Z':-1}) * grav**2 * cs**-2

##########################################    
        diaTdz = layers_apply((rho_th*DibaT).where(ds.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                              yend+2*12**-1)
                                                                    ).chunk({'nmemb':membchunk,'Z':-1,
                                                                             'YC':ychunk,'XC':xchunk}),
                              (rho_insitu-rhoConst),
                              siglayers, fine_drf, fine_drc, fine_fact=nremap,
                              lev_name='Z', Tlev_name='sig').compute()
        diaSdz = layers_apply((rho_s*DibaS).where(ds.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                             yend+2*12**-1)
                                                                   ).chunk({'nmemb':membchunk,'Z':-1,
                                                                            'YC':ychunk,'XC':xchunk}),
                              (rho_insitu-rhoConst),
                              siglayers, fine_drf, fine_drc, fine_fact=nremap,
                              lev_name='Z', Tlev_name='sig').compute()
        cmpdz = layers_apply(comp.where(ds.maskC!=0.).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                         ).chunk({'nmemb':membchunk,
                                                                  'YC':ychunk,'XC':xchunk}),
                             (rho_insitu-rhoConst),
                             siglayers, fine_drf, fine_drc, fine_fact=nremap,
                             lev_name='Z', Tlev_name='sig').compute()       
##########################################
        varpidz = (-grav*(diaTdz + diaSdz)*rhoConst**-1 
                   + cmpdz
                  )
        dsave = varpidz.to_dataset(name='varpidz')
        dsave.coords['sigp1'] = ('sigp1',sigp1layers)
#         dsave.to_netcdf(op.join(savedir,'%4d/varpidz-compressible_membs_parallel-%2d_%010d.nc' 
#                                 % (year,nremap,itime)))
        dsave.to_zarr(op.join(savedir,'%4d/Varpi/%07d/' % (year,itime)), mode='w')
        dsave.close()

        del varpidz, diaTdz, diaSdz, cmpdz
        print("Varpi saved.")

##########################################
        dpdx = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.),'X') 
                * ds.dxC**-1
               ).where(ds.maskW!=0.)
        dpdy = (grid.diff(ds.PHIHYD.where(ds.maskC!=0.),'Y',boundary='fill') 
                * ds.dyC**-1
               ).where(ds.maskS!=0.)   

        pxdz = layers_apply(dpdx.sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                    ).chunk({'nmemb':membchunk,'YC':ychunk,'XG':xchunk}),
                            (rhox-rhoConst),
                            siglayers, fine_drf, fine_drc, fine_fact=nremap,
                            lev_name='Z', Tlev_name='sig').compute()
        pydz = layers_apply(dpdy.sel(YG=slice(ystart-12**-1,yend+2*12**-1)
                                    ).chunk({'nmemb':membchunk,'YG':ychunk,'XC':xchunk}),
                            (rhoy-rhoConst),
                            siglayers, fine_drf, fine_drc, fine_fact=nremap,
                            lev_name='Z', Tlev_name='sig').compute()
        tdz = layers_apply(theta.where(ds.maskC!=0.).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                        ).chunk({'nmemb':membchunk,
                                                                 'YC':ychunk,'XC':xchunk}),
                           (rho_insitu-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
        sdz = layers_apply(salt.where(ds.maskC!=0.).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                       ).chunk({'nmemb':membchunk,
                                                                'YC':ychunk,'XC':xchunk}),
                           (rho_insitu-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
        udz = layers_apply(ds.UVEL.where(ds.maskW!=0.).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                          ).chunk({'nmemb':membchunk,
                                                                   'YC':ychunk,'XG':xchunk}),
                           (rhox-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()
        vdz = layers_apply(ds.VVEL.where(ds.maskS!=0.).sel(YG=slice(ystart-12**-1,yend+2*12**-1)
                                                          ).chunk({'nmemb':membchunk,
                                                                   'YG':ychunk,'XC':xchunk}),
                           (rhoy-rhoConst),
                           siglayers, fine_drf, fine_drc, fine_fact=nremap,
                           lev_name='Z', Tlev_name='sig').compute()

        del rho_insitu, rhoz, rhoy, rhox
        end = time.time()
        print("Dyns interpolated. Lapse:", end-start)

##########################################   
        dsave = vdz.to_dataset(name='vdz')
        dsave['udz'] = udz
        dsave['tdz'] = tdz
        dsave['sdz'] = sdz
        dsave['pxdz'] = pxdz
        dsave['pydz'] = pydz
        dsave.coords['sigp1'] = ('sigp1',sigp1layers)
#         dsave.to_netcdf(op.join(savedir,'%4d/Dyns_membs_parallel-%2d_%010d.nc' 
#                                 % (year,nremap,itime)))
        dsave.to_zarr(op.join(savedir,'%4d/Dyns/%07d/' % (year,itime)), mode='w')
        dsave.close()

        print("Dyn saved.")
        del tdz, sdz, vdz, udz, pydz, pxdz
        ds.close()

        print(itime)

Zs interpolated. Lapse: 564.3711264133453
Zs saved.
Varpi saved.
Dyns interpolated. Lapse: 928.3569619655609
Dyn saved.
1360800
Zs interpolated. Lapse: 610.0435085296631
Zs saved.
Varpi saved.
Dyns interpolated. Lapse: 896.6788005828857
Dyn saved.
1362960
Zs interpolated. Lapse: 616.134152173996
Zs saved.
Varpi saved.
